In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print(f'User uploaded file "{fn}" with length {len(uploaded[fn])} bytes')

# Fintech Credit Risk & Lending Analytics  
## Data Understanding and Cleaning

### Objective
The objective of this notebook is to understand, clean, and prepare borrower-level credit data for building a fintech-style credit risk and lending decision system.  
This step ensures that the data accurately reflects real-world lending scenarios before any modeling is performed.

---

## Dataset Description

The dataset represents an anonymized retail credit portfolio similar to those used by banks and fintech lenders.  
Each observation corresponds to an individual borrower, with features capturing income, debt burden, credit utilization, and past repayment behavior.

The target variable indicates whether a borrower experienced **serious delinquency (default)** within a two-year horizon.

---

## Target Variable

### `SeriousDlqin2yrs`
- Binary indicator of loan default
- `1` → Borrower was 90+ days delinquent (default)
- `0` → Borrower did not default

This variable represents **credit risk realization** and is the primary outcome modeled in this project.

---

## Key Credit Risk Features

### 1. Income and Debt Capacity

#### `MonthlyIncome`
- Borrower’s reported monthly income
- Lower income levels are associated with higher default risk due to reduced repayment capacity

#### `DebtRatio`
- Ratio of total monthly debt obligations to monthly income
- Higher values indicate financial stress and leverage

These variables capture the **ability-to-pay** dimension of credit risk.

---

### 2. Credit Utilization

#### `RevolvingUtilizationOfUnsecuredLines`
- Proportion of available unsecured credit currently being used
- High utilization indicates liquidity stress and increased default probability

This is a key indicator of **short-term financial pressure**.

---

### 3. Past Repayment Behavior (Most Important)

The strongest predictors of future default are historical delinquencies:

- `NumberOfTime30-59DaysPastDueNotWorse`
- `NumberOfTime60-89DaysPastDueNotWorse`
- `NumberOfTimes90DaysLate`

These variables capture prior repayment discipline and are heavily weighted in real-world credit scoring systems.

---

### 4. Credit Exposure

#### `NumberOfOpenCreditLinesAndLoans`
- Total number of active credit accounts
- Higher values indicate greater exposure and repayment obligations

#### `NumberRealEstateLoansOrLines`
- Number of mortgage or real-estate-backed loans
- Represents long-term leverage

---

### 5. Demographic and Household Factors

#### `age`
- Borrower age
- Credit risk often exhibits non-linear patterns across age groups

#### `NumberOfDependents`
- Number of dependents supported by the borrower
- Higher values increase financial burden

---

## Data Quality Issues

Real-world credit data often contains:
- Missing income values
- Implausible ages
- Extreme outliers

These issues must be addressed to ensure reliable modeling and realistic risk estimates.

---

## Data Cleaning Strategy

The following cleaning steps are applied:

1. Removal of identifier columns with no predictive value  
2. Treatment of missing income using median imputation  
3. Removal of unrealistic borrower ages  
4. Basic sanity checks on extreme values  

These steps reflect standard preprocessing practices used in banking and fintech analytics.

---

## Output of This Notebook

The output of this notebook is a **cleaned and structured dataset** suitable for:
- Exploratory risk analysis
- Credit risk modeling
- Lending decision simulation

Subsequent notebooks build directly on this cleaned dataset.


In [2]:
import pandas as pd
import numpy as np

# Load training data
df = pd.read_csv("cs-training.csv")

print("Shape of raw data:", df.shape)
df.head()


Shape of raw data: (150000, 12)


,Unnamed: 0,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
0,1,1,0.766127,45,2,0.802982,9120.0,13,0,6,0,2.0
1,2,0,0.957151,40,0,0.121876,2600.0,4,0,0,0,1.0
2,3,0,0.658180,38,1,0.085113,3042.0,2,1,0,0,0.0
3,4,0,0.233810,30,0,0.036050,3300.0,5,0,0,0,0.0
4,5,0,0.907239,49,1,0.024926,63588.0,7,0,1,0,0.0


In [3]:
df = df.drop(columns=["Unnamed: 0"])


In [4]:
df = df.rename(columns={"SeriousDlqin2yrs": "default"})


In [5]:
df.info()
df.describe()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 11 columns):
 #   Column                                Non-Null Count   Dtype  
---  ------                                --------------   -----  
 0   default                               150000 non-null  int64  
 1   RevolvingUtilizationOfUnsecuredLines  150000 non-null  float64
 2   age                                   150000 non-null  int64  
 3   NumberOfTime30-59DaysPastDueNotWorse  150000 non-null  int64  
 4   DebtRatio                             150000 non-null  float64
 5   MonthlyIncome                         120269 non-null  float64
 6   NumberOfOpenCreditLinesAndLoans       150000 non-null  int64  
 7   NumberOfTimes90DaysLate               150000 non-null  int64  
 8   NumberRealEstateLoansOrLines          150000 non-null  int64  
 9   NumberOfTime60-89DaysPastDueNotWorse  150000 non-null  int64  
 10  NumberOfDependents                    146076 non-null  float64
dtype

,default,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
count,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000,1.202690e+05,150000.000000,150000.000000,150000.000000,150000.000000,146076.000000
mean,0.066840,6.048438,52.295207,0.421033,353.005076,6.670221e+03,8.452760,0.265973,1.018240,0.240387,0.757222
std,0.249746,249.755371,14.771866,4.192781,2037.818523,1.438467e+04,5.145951,4.169304,1.129771,4.155179,1.115086
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.029867,41.000000,0.000000,0.175074,3.400000e+03,5.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.154181,52.000000,0.000000,0.366508,5.400000e+03,8.000000,0.000000,1.000000,0.000000,0.000000
75%,0.000000,0.559046,63.000000,0.000000,0.868254,8.249000e+03,11.000000,0.000000,2.000000,0.000000,1.000000
max,1.000000,50708.000000,109.000000,98.000000,329664.000000,3.008750e+06,58.000000,98.000000,54.000000,98.000000,20.000000


In [6]:
# Check missing values
df.isnull().sum()


,0
default,0
RevolvingUtilizationOfUnsecuredLines,0
age,0
NumberOfTime30-59DaysPastDueNotWorse,0
DebtRatio,0
MonthlyIncome,29731
NumberOfOpenCreditLinesAndLoans,0
NumberOfTimes90DaysLate,0
NumberRealEstateLoansOrLines,0
NumberOfTime60-89DaysPastDueNotWorse,0


In [7]:
# Median imputation for income
df["MonthlyIncome"] = df["MonthlyIncome"].fillna(df["MonthlyIncome"].median())

# Missing dependents treated as zero
df["NumberOfDependents"] = df["NumberOfDependents"].fillna(0)


In [8]:
df = df[(df["age"] >= 18) & (df["age"] <= 100)]


In [9]:
delinq_cols = [
    "NumberOfTime30-59DaysPastDueNotWorse",
    "NumberOfTime60-89DaysPastDueNotWorse",
    "NumberOfTimes90DaysLate"
]

for col in delinq_cols:
    df[col] = np.where(df[col] > 10, 10, df[col])


In [10]:
print("Shape after cleaning:", df.shape)
df.isnull().sum()


Shape after cleaning: (149986, 11)


,0
default,0
RevolvingUtilizationOfUnsecuredLines,0
age,0
NumberOfTime30-59DaysPastDueNotWorse,0
DebtRatio,0
MonthlyIncome,0
NumberOfOpenCreditLinesAndLoans,0
NumberOfTimes90DaysLate,0
NumberRealEstateLoansOrLines,0
NumberOfTime60-89DaysPastDueNotWorse,0


In [11]:
df.to_csv("cleaned_credit.csv", index=False)
